# Improved Hangman Agent

This notebook implements an improved agent that focuses on the HMM's pattern-matching capabilities.

## Key Improvements:
1. **Pure HMM Strategy**: Relies primarily on HMM probability distributions
2. **Better Pattern Matching**: Leverages word filtering more effectively
3. **Simpler Architecture**: Removes complex Q-learning for more reliable performance

In [ ]:
import numpy as np
import pickle
import random
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

sns.set_style('whitegrid')
%matplotlib inline

## 1. Load the HMM Model

In [ ]:
# Import the PositionalHMM class definition
class PositionalHMM:
    """A simplified HMM using positional letter frequencies"""
    
    def __init__(self):
        self.letter_freq_by_length = {}
        self.overall_freq_by_length = {}
        self.word_list_by_length = {}
        self.all_letters = set('abcdefghijklmnopqrstuvwxyz')
        
    def get_letter_probabilities(self, masked_word, guessed_letters):
        """Get probability distribution over remaining letters"""
        length = len(masked_word)
        remaining_letters = self.all_letters - set(guessed_letters)
        
        if length not in self.word_list_by_length:
            return self._get_default_probabilities(remaining_letters)
        
        # Filter words matching the pattern
        matching_words = self._get_matching_words(masked_word, guessed_letters, length)
        
        if not matching_words:
            return self._get_positional_probabilities(masked_word, remaining_letters, length)
        
        # Count letter frequencies in matching words
        letter_counts = Counter()
        for word in matching_words:
            for letter in set(word):
                if letter in remaining_letters:
                    letter_counts[letter] += 1
        
        total = sum(letter_counts.values())
        if total == 0:
            return self._get_default_probabilities(remaining_letters)
        
        probabilities = {letter: count / total for letter, count in letter_counts.items()}
        
        for letter in remaining_letters:
            if letter not in probabilities:
                probabilities[letter] = 1e-6
        
        return probabilities
    
    def _get_matching_words(self, masked_word, guessed_letters, length):
        """Find words matching the current pattern"""
        matching_words = []
        for word in self.word_list_by_length[length]:
            if self._matches_pattern(word, masked_word, guessed_letters):
                matching_words.append(word)
        return matching_words
    
    def _matches_pattern(self, word, masked_word, guessed_letters):
        """Check if a word matches the masked pattern"""
        if len(word) != len(masked_word):
            return False
        
        for i, (w_char, m_char) in enumerate(zip(word, masked_word)):
            if m_char != '_':
                if w_char != m_char:
                    return False
            else:
                if w_char in guessed_letters:
                    return False
        return True
    
    def _get_positional_probabilities(self, masked_word, remaining_letters, length):
        """Get probabilities based on positional frequencies"""
        letter_scores = defaultdict(float)
        
        for pos, char in enumerate(masked_word):
            if char == '_':
                if pos in self.letter_freq_by_length[length]:
                    pos_freq = self.letter_freq_by_length[length][pos]
                    total = sum(pos_freq.values())
                    if total > 0:
                        for letter in remaining_letters:
                            letter_scores[letter] += pos_freq[letter] / total
        
        total_score = sum(letter_scores.values())
        if total_score == 0:
            return self._get_default_probabilities(remaining_letters)
        
        return {letter: score / total_score for letter, score in letter_scores.items()}
    
    def _get_default_probabilities(self, remaining_letters):
        """Default English letter frequencies"""
        default_freq = {
            'e': 0.127, 't': 0.091, 'a': 0.082, 'o': 0.075, 'i': 0.070,
            'n': 0.067, 's': 0.063, 'h': 0.061, 'r': 0.060, 'd': 0.043,
            'l': 0.040, 'c': 0.028, 'u': 0.028, 'm': 0.024, 'w': 0.024,
            'f': 0.022, 'g': 0.020, 'y': 0.020, 'p': 0.019, 'b': 0.015,
            'v': 0.010, 'k': 0.008, 'j': 0.002, 'x': 0.002, 'q': 0.001, 'z': 0.001
        }
        
        probabilities = {}
        for letter in remaining_letters:
            probabilities[letter] = default_freq.get(letter, 1e-6)
        
        total = sum(probabilities.values())
        return {letter: prob / total for letter, prob in probabilities.items()}

# Load the trained HMM
with open('hmm_model.pkl', 'rb') as f:
    hmm = pickle.load(f)

print("HMM model loaded successfully!")

## 2. Create Pure HMM Agent

In [ ]:
class ImprovedHangmanAgent:
    """Pure HMM-based agent for Hangman"""
    
    def __init__(self, hmm_model):
        self.hmm = hmm_model
    
    def get_action(self, masked_word, guessed_letters):
        """Select the best letter based on HMM probabilities"""
        # Get HMM probabilities
        probabilities = self.hmm.get_letter_probabilities(masked_word, guessed_letters)
        
        # Choose the letter with highest probability
        if probabilities:
            return max(probabilities.items(), key=lambda x: x[1])[0]
        else:
            # Fallback: choose from remaining letters
            all_letters = set('abcdefghijklmnopqrstuvwxyz')
            remaining = list(all_letters - set(guessed_letters))
            return random.choice(remaining) if remaining else 'e'

# Create the improved agent
agent = ImprovedHangmanAgent(hmm)
print("Improved agent created!")

## 3. Load Test Data

In [ ]:
# Load test data
with open('Data/Data/test.txt', 'r') as f:
    test_words = [line.strip().lower() for line in f if line.strip()]

print(f"Loaded {len(test_words)} test words")

## 4. Test on Sample Words

In [ ]:
def play_game(agent, target_word, max_lives=6, verbose=False):
    """Play a single game of Hangman"""
    target_word = target_word.lower()
    masked_word = '_' * len(target_word)
    guessed_letters = set()
    lives = max_lives
    wrong_guesses = 0
    repeated_guesses = 0
    
    if verbose:
        print(f"\nTarget: {target_word}")
        print(f"Starting: {masked_word}")
    
    step = 0
    while '_' in masked_word and lives > 0:
        step += 1
        
        # Get agent's guess
        guess = agent.get_action(masked_word, guessed_letters)
        
        # Check for repeated guess
        if guess in guessed_letters:
            repeated_guesses += 1
            if verbose:
                print(f"Step {step}: Repeated guess '{guess}'!")
            continue
        
        guessed_letters.add(guess)
        
        # Check if guess is correct
        if guess in target_word:
            # Update masked word
            new_masked = list(masked_word)
            for i, char in enumerate(target_word):
                if char == guess:
                    new_masked[i] = guess
            masked_word = ''.join(new_masked)
            if verbose:
                print(f"Step {step}: {masked_word} | Guess '{guess}' ✓ | Lives: {lives}")
        else:
            lives -= 1
            wrong_guesses += 1
            if verbose:
                print(f"Step {step}: {masked_word} | Guess '{guess}' ✗ | Lives: {lives}")
    
    won = '_' not in masked_word
    if verbose:
        print(f"\nResult: {'WON' if won else 'LOST'} | Wrong: {wrong_guesses} | Repeated: {repeated_guesses}")
    
    return {
        'won': won,
        'wrong_guesses': wrong_guesses,
        'repeated_guesses': repeated_guesses,
        'steps': step
    }

# Test on sample words
sample_words = ['python', 'machine', 'learning', 'algorithm', 'neural']
print("Testing on sample words:\n")
print("="*60)

for word in sample_words:
    result = play_game(agent, word, verbose=True)
    print("="*60)

## 5. Evaluate on Full Test Set

In [ ]:
def evaluate_agent(agent, test_words, num_games=2000, max_lives=6):
    """Evaluate the agent on test words"""
    print(f"Evaluating agent on {num_games} games...\n")
    
    # Sample games
    if len(test_words) < num_games:
        eval_words = random.choices(test_words, k=num_games)
    else:
        eval_words = random.sample(test_words, num_games)
    
    # Statistics
    wins = 0
    total_wrong_guesses = 0
    total_repeated_guesses = 0
    game_lengths = []
    wrong_guesses_per_game = []
    
    for word in tqdm(eval_words):
        result = play_game(agent, word, max_lives=max_lives)
        
        if result['won']:
            wins += 1
        
        total_wrong_guesses += result['wrong_guesses']
        total_repeated_guesses += result['repeated_guesses']
        game_lengths.append(result['steps'])
        wrong_guesses_per_game.append(result['wrong_guesses'])
    
    # Calculate metrics
    success_rate = wins / num_games
    avg_wrong_guesses = total_wrong_guesses / num_games
    avg_repeated_guesses = total_repeated_guesses / num_games
    avg_game_length = np.mean(game_lengths)
    
    # Calculate final score
    final_score = (success_rate * num_games) - (total_wrong_guesses * 5) - (total_repeated_guesses * 2)
    
    return {
        'num_games': num_games,
        'wins': wins,
        'success_rate': success_rate,
        'total_wrong_guesses': total_wrong_guesses,
        'total_repeated_guesses': total_repeated_guesses,
        'avg_wrong_guesses': avg_wrong_guesses,
        'avg_repeated_guesses': avg_repeated_guesses,
        'avg_game_length': avg_game_length,
        'final_score': final_score,
        'game_lengths': game_lengths,
        'wrong_guesses_per_game': wrong_guesses_per_game
    }

# Run evaluation
results = evaluate_agent(agent, test_words, num_games=2000, max_lives=6)

## 6. Display Results

In [ ]:
# Display results
print("\n" + "="*60)
print(" " * 15 + "IMPROVED AGENT RESULTS")
print("="*60)
print(f"\nTotal Games Played: {results['num_games']}")
print(f"\n{'Metric':<35} {'Value':>20}")
print("-"*60)
print(f"{'Wins':<35} {results['wins']:>20}")
print(f"{'Success Rate':<35} {results['success_rate']:>19.2%}")
print(f"{'Total Wrong Guesses':<35} {results['total_wrong_guesses']:>20}")
print(f"{'Total Repeated Guesses':<35} {results['total_repeated_guesses']:>20}")
print(f"{'Avg Wrong Guesses per Game':<35} {results['avg_wrong_guesses']:>20.2f}")
print(f"{'Avg Repeated Guesses per Game':<35} {results['avg_repeated_guesses']:>20.2f}")
print(f"{'Avg Game Length (guesses)':<35} {results['avg_game_length']:>20.2f}")
print("\n" + "="*60)
print(f"{'FINAL SCORE':<35} {results['final_score']:>20.2f}")
print("="*60)

print("\n📊 Score Breakdown:")
print(f"  Success Rate × 2000 = {results['success_rate'] * 2000:.2f}")
print(f"  Wrong Guesses × 5 = -{results['total_wrong_guesses'] * 5:.2f}")
print(f"  Repeated Guesses × 2 = -{results['total_repeated_guesses'] * 2:.2f}")
print(f"  " + "-"*40)
print(f"  Final Score = {results['final_score']:.2f}")

## 7. Visualize Results

In [ ]:
# Create visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Win/Loss distribution
wins_losses = [results['wins'], results['num_games'] - results['wins']]
colors = ['#2ecc71', '#e74c3c']
axes[0, 0].pie(wins_losses, labels=['Wins', 'Losses'], autopct='%1.1f%%', 
               colors=colors, startangle=90)
axes[0, 0].set_title(f"Win/Loss Distribution\n(Success Rate: {results['success_rate']:.2%})")

# 2. Distribution of wrong guesses
axes[0, 1].hist(results['wrong_guesses_per_game'], bins=range(0, 8), 
                color='steelblue', alpha=0.7, edgecolor='black')
axes[0, 1].axvline(results['avg_wrong_guesses'], color='red', 
                   linestyle='--', label=f"Mean: {results['avg_wrong_guesses']:.2f}")
axes[0, 1].set_xlabel('Wrong Guesses per Game')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title('Distribution of Wrong Guesses')
axes[0, 1].legend()
axes[0, 1].grid(axis='y', alpha=0.3)

# 3. Distribution of game lengths
axes[1, 0].hist(results['game_lengths'], bins=30, color='green', alpha=0.7, edgecolor='black')
axes[1, 0].axvline(results['avg_game_length'], color='red', 
                   linestyle='--', label=f"Mean: {results['avg_game_length']:.2f}")
axes[1, 0].set_xlabel('Game Length (Number of Guesses)')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title('Distribution of Game Lengths')
axes[1, 0].legend()
axes[1, 0].grid(axis='y', alpha=0.3)

# 4. Score breakdown
categories = ['Success\nReward', 'Wrong\nGuess\nPenalty', 'Repeated\nGuess\nPenalty', 'Final\nScore']
values = [
    results['success_rate'] * 2000,
    -results['total_wrong_guesses'] * 5,
    -results['total_repeated_guesses'] * 2,
    results['final_score']
]
colors_bar = ['green', 'red', 'orange', 'blue']
bars = axes[1, 1].bar(categories, values, color=colors_bar, alpha=0.7, edgecolor='black')
axes[1, 1].set_ylabel('Score Contribution')
axes[1, 1].set_title('Score Breakdown')
axes[1, 1].axhline(0, color='black', linewidth=0.8)
axes[1, 1].grid(axis='y', alpha=0.3)

# Add value labels on bars
for bar, value in zip(bars, values):
    height = bar.get_height()
    axes[1, 1].text(bar.get_x() + bar.get_width()/2., height,
                    f'{value:.0f}',
                    ha='center', va='bottom' if height > 0 else 'top')

plt.tight_layout()
plt.savefig('improved_results.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Visualization saved as 'improved_results.png'")

## 8. Save Results

In [ ]:
# Save results
with open('improved_results.txt', 'w', encoding='utf-8') as f:
    f.write("="*60 + "\n")
    f.write(" " * 15 + "IMPROVED AGENT EVALUATION RESULTS\n")
    f.write("="*60 + "\n\n")
    
    f.write(f"Total Games Played: {results['num_games']}\n")
    f.write(f"Max Lives per Game: 6\n\n")
    
    f.write(f"{'Metric':<40} {'Value':>15}\n")
    f.write("-"*60 + "\n")
    f.write(f"{'Wins':<40} {results['wins']:>15}\n")
    f.write(f"{'Success Rate':<40} {results['success_rate']:>14.2%}\n")
    f.write(f"{'Total Wrong Guesses':<40} {results['total_wrong_guesses']:>15}\n")
    f.write(f"{'Total Repeated Guesses':<40} {results['total_repeated_guesses']:>15}\n")
    f.write(f"{'Avg Wrong Guesses per Game':<40} {results['avg_wrong_guesses']:>15.3f}\n")
    f.write(f"{'Avg Repeated Guesses per Game':<40} {results['avg_repeated_guesses']:>15.3f}\n")
    f.write(f"{'Avg Game Length (guesses)':<40} {results['avg_game_length']:>15.2f}\n")
    
    f.write("\n" + "="*60 + "\n")
    f.write(f"{'FINAL SCORE':<40} {results['final_score']:>15.2f}\n")
    f.write("="*60 + "\n\n")
    
    f.write("Score Calculation:\n")
    f.write(f"  Success Rate × 2000 = {results['success_rate'] * 2000:.2f}\n")
    f.write(f"  Wrong Guesses × 5 = -{results['total_wrong_guesses'] * 5:.2f}\n")
    f.write(f"  Repeated Guesses × 2 = -{results['total_repeated_guesses'] * 2:.2f}\n")
    f.write(f"  " + "-"*40 + "\n")
    f.write(f"  Final Score = {results['final_score']:.2f}\n")

# Save the improved agent
with open('improved_agent.pkl', 'wb') as f:
    pickle.dump(agent, f)

print("\n✓ Results saved to 'improved_results.txt'")
print("✓ Agent saved to 'improved_agent.pkl'")

## Summary

This improved agent uses a **pure HMM-based strategy** that:

1. **Relies on Pattern Matching**: Filters corpus words matching the current game state
2. **Uses Probability Distributions**: Always selects the letter with highest probability
3. **No Complex Training**: Doesn't require Q-learning training, making it more reliable
4. **Better Performance**: Should achieve significantly higher success rates

### Expected Improvements:
- Higher win rate (60-80% vs 20%)
- Fewer wrong guesses per game
- Positive final score
- More consistent performance